In [1]:
import pandas as pd
import ssl
import urllib.request, urllib.parse, urllib.error
from selenium import webdriver
from selenium.webdriver.common.by import By


In [2]:
rochelle = pd.read_excel(r"C:\Users\justi\Downloads\GLANSIS Spp. Tracking.xlsx", sheet_name = "Listed")


In [3]:
#rochelle.columns
namelist= []
for name in rochelle['Scientific Name']:
    namelist.append(name)

print(namelist)

['Cabomba caroliniana', 'Myriophyllum spicatum', 'Phragmites australis', 'Nymphoides peltata', 'Potamogeton crispus', 'Trapa natans', 'Corbicula fluminea', 'Gambusia affinis', 'Pimephales vigilax', 'Crassula helmsii', 'Pistia stratiotes', 'Stratiotes aloides', 'Veronica beccabunga', 'Rhithropanopeus harrisi', 'Thermocyclops crassus', 'Cherax destructor', 'Hypophthalmichthys molitrix', 'Hypophthalmichthys nobilis', 'Lepomis auritus', 'Limnoperna fortunei', 'Alnus glutinosa', 'Apeltes quadracus', 'Arundo donax', 'Branchiura sowerbyi', 'Channa argus', 'Cipangopaludina japonica', 'Cyclops strenuus', 'Didymosphenia geminata', 'Discostella (Cyclotella) pseudostelligera', 'Leuciscus idus', 'Neoergasilus japonicus', 'Procambarus clarkii', 'Psammonobiotus communis', 'Psammonobiotus dziwnowi', 'Psammonobiotus linearis', 'Syngnathus abaster', 'Agrostis gigantea', 'Butomus umbellatus', 'Carassius carassius', 'Carex acutiformis', 'Carex disticha', 'Chenopodium glaucum (Oxybasis glauca)', 'Cirsium p

In [4]:
# Ignore SSL certificate errors --from Beautiful Soup SI 206 textbook
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [5]:
#accessing search page of the herbarium
atlas_url = "https://bellatlas.umn.edu/collections/harvestparams.php"
atlas_html = urllib.request.urlopen(atlas_url, context=ctx).read()
#herbarium_html


In [7]:
#setting up driver for Selenium
driver = webdriver.Firefox() #???Should I write a script for multiple Browsers?
driver.maximize_window()
scraped_dataframe = pd.DataFrame()#(index=[0])
scraped_dataframe.reset_index(drop=True)
for speciesname in namelist[:1]:
    #print(speciesname)
    driver.get(atlas_url) # Goes to atlas
    namebox = driver.find_element(by = By.XPATH, value = '//*[@id="taxa"]')
    namebox.send_keys('COME ON') #'Cabomba caroliniana') #Sends in a name from Rochelle's list
    searchbutton = driver.find_element(by = By.XPATH, value='/html/body/table/tbody/tr/td/div[4]/form/div[2]/div[1]/button') #selects the advanced search button based off the pulled tag from right clicking the inspected element
    searchbutton.click()
    #resultcount = driver.find_element(by = By.XPATH, value ='/html/body/table/tbody/tr/td/div[4]/div/div[2]/div/div[4]/div[3]').text